**Part 1: Read and Process Data**

In [9]:
from glob import glob
from pathlib import Path
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
#to access the files
all_file_path = glob('/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/*.edf')
print(len(all_file_path))

28


In [6]:
all_file_path[0]

'/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s01.edf'

In [11]:
print(all_file_path)


['/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s01.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h01.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h14.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s14.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s02.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h02.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h03.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s03.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s07.edf', '/Users/folasewaabdulsalam/Signal_Processing/

In [13]:


# Splitting the data file path into healthy and schizophrenia patients

healthy_file_path = [i for i in all_file_path if 'h' in i.split('/')[-1]]  # Extract filename using '/'
schizo_patient_file_path = [i for i in all_file_path if 's' in i.split('/')[-1]]  # Extract filename using '/'

print(len(healthy_file_path), len(schizo_patient_file_path))


14 14


In [14]:
#a function to read the path and get the data from it
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference() #set the channel references, this line of code will get set the reference as an average of the channels
    data.filter(l_freq = 0.5, h_freq = 45) #setting low frequency as 0.5 and high frequency as 45
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap = 1) #split the data into segments
    array = epochs.get_data() #this will convert the data to a numpy array
    return array



In [15]:
#let us read a data
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
N

In [16]:
sample_data.shape #231 - no of epochs, 19 - no of channels, 1250, length of signal

(231, 19, 1250)

In [18]:
import sys
import os

# Suppress output
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout
# Wrap in SuppressOutput context manager
with SuppressOutput(): #mainly using %%capture could have worked if I were using a jupyter kernel, but since it is on vscode, I had to create a function called suppress outout and wrap my lines of code
    #let's read all the files, the use of capture is to not allow those information above to print for all the data
    healthy_epochs_array = [read_data(i) for i in healthy_file_path]
    schizo_epochs_array = [read_data(i) for i in schizo_patient_file_path]

In [19]:
healthy_epochs_array[0].shape

(231, 19, 1250)

In [20]:
#creating a label 0 for healthy, 1 for schizo
healthy_epochs_labels = [len(i)*[0] for i in healthy_epochs_array]
schizo_epochs_labels = [len(i)*[1] for i in schizo_epochs_array]
len(healthy_epochs_labels), len(schizo_epochs_labels)

(14, 14)

In [21]:
#combining both of the files - the data array and the label
data_list = healthy_epochs_array + schizo_epochs_array
label_list = healthy_epochs_labels + schizo_epochs_labels


In [22]:
#splitting the data based on subject and not on epochs or channels, therefore we assign a group to each subject making 28 groups
group_list =[[i]*len(j) for i, j in enumerate(data_list)]
len(group_list)

28

In [23]:
data_array = np.vstack(data_list) #combines array vertically
label_array = np.hstack(label_list) #combines array horizontally
group_array = np.hstack(group_list)

print(data_array.shape, label_array.shape, group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


In [25]:
#we want to get different features from this data
from scipy import stats

def mean(x):
    return np.mean(x, axis = -1)

def std(x):
    return np.std(x, axis = -1)

def ptp(x):
    return np.ptp(x, axis = -1)

def var(x):
    return np.var(x, axis = -1)

def minim(x):
    return np.min(x, axis = -1)

def maxim(x):
    return np.max(x, axis = -1)

def argminim(x):
    return np.argmin(x, axis = -1)

def argmaxim(x):
    return np.argmax(x, axis = -1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis = -1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis = -1)), axis = -1)

def skewness(x):
    return stats.skew(x, axis = -1)

def kurtosis(x):
    return stats.kurtosis(x, axis = -1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), 
                          argmaxim(x), argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis = -1)



In [26]:
#let us run a loop to extract these features

extracted_features = []

for data in data_array:
    extracted_features.append(concatenate_features(data))

In [27]:
extracted_features_array = np.array(extracted_features)
extracted_features_array.shape


(7201, 228)

**Part 2: Machine Learning Classification**

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV




In [35]:
logReg = LogisticRegression()
grpKfold = GroupKFold(5)
pipe_line = Pipeline([('scaler', StandardScaler()), ('logReg', logReg)])
param_grid = {'logReg__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]}
grid_search = GridSearchCV(pipe_line, param_grid, cv = grpKfold, n_jobs=12)
grid_search.fit(extracted_features_array, label_array, groups = group_array)


/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

GridSearchCV(cv=GroupKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logReg', LogisticRegression())]),
             n_jobs=12, param_grid={'logReg__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [36]:
grid_search.best_score_

np.float64(0.6564742404858336)

Part 3: Deep Learning with CNN

In [37]:
#checking the shape of each array
data_array.shape, label_array.shape, group_array.shape #1250 is the length, 19 is the no of channels, 7201 is the no of segment

((7201, 19, 1250), (7201,), (7201,))

In [40]:
#cnn requires the channel to be at the end, so we move the channel to the end

data_array = np.moveaxis(data_array, 1, 2)
data_array.shape

(7201, 1250, 19)

In [45]:
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPool1D, GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

def cnnmodel():
    clear_session()
    model = Sequential()
    model.add(Conv1D(filters = 5, kernel_size = 3, strides = 1, input_shape = (1250, 19))) #input layer
    model.add(BatchNormalization())
    model.add(LeakyReLU()) #activation function
    model.add(MaxPool1D(pool_size = 2, strides = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 5, kernel_size = 3, strides = 1)) #2nd layer
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size = 2, strides = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 5, kernel_size = 3, strides = 1)) #3rd layer
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size = 2, strides = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 5, kernel_size = 3, strides = 1)) #4th layer
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size = 2, strides = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 5, kernel_size = 3, strides = 1)) #output layer
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1, activation = 'sigmoid')) #dense is 1 because we are classifying whether the person has schizo or not, label 0 or 1

    model.compile('adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

model = cnnmodel()
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1248, 5)        │           290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1248, 5)        │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1248, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 624, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 624, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 622, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 622, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 311, 5)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 311, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 309, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 309, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_1             │ (None, 154, 5)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 154, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 152, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 152, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_2             │ (None, 76, 5)          │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 76, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 74, 5)          │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 74, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 5)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 636 (2.48 KB)

 Trainable params: 626 (2.45 KB)

 Non-trainable params: 10 (40.00 B)

In [64]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut #for cross validation
from sklearn.preprocessing import StandardScaler #to scale the dataset



In [65]:
#manually reserve a test group from the validation group

unique_groups = np.unique(group_array)

#reserve one group for testing
test_group = unique_groups[-1] #last group
test_indices = np.where(group_array ==test_group)[0]

#create the test set
test_features  = data_array[test_indices]
test_labels = label_array[test_indices]

#creating training + validation set

train_val_indices = np.where(group_array != test_group)[0]
train_val_features = data_array[train_val_indices]
train_val_labels = label_array[train_val_indices] 
train_val_groups = group_array[train_val_indices]



In [66]:
accuracy = []
grpKfold = GroupKFold(n_splits=5)  # 5-fold cross-validation

for train_index, val_index in grpKfold.split(train_val_features, train_val_labels, groups=train_val_groups):
    train_features, train_labels = train_val_features[train_index], train_val_labels[train_index]
    val_features, val_labels = train_val_features[val_index], train_val_labels[val_index]

    # Standardize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)

    # Train model
    model = cnnmodel()
    model.fit(train_features, train_labels, epochs=10, batch_size=100, validation_data=(val_features, val_labels))

    # Append validation accuracy
    accuracy.append(model.evaluate(val_features, val_labels)[1])  # accuracy is at index 1

# Print cross-validation results
print(f"Cross-Validation Average Accuracy: {np.mean(accuracy) * 100:.2f}%")


/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.5107 - loss: 0.7034 - val_accuracy: 0.6853 - val_loss: 0.6748
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.5004 - loss: 0.6926 - val_accuracy: 0.7992 - val_loss: 0.6760
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.5697 - loss: 0.6869 - val_accuracy: 0.6121 - val_loss: 0.6584
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.6763 - loss: 0.6364 - val_accuracy: 0.6322 - val_loss: 0.6158
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.7759 - loss: 0.5146 - val_accuracy: 0.7039 - val_loss: 0.5772
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.7768 - loss: 0.4918 - val_accuracy: 0.7709 - val_loss: 0.5139
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7852 - loss: 0.4702 - val_accuracy: 0.7026 - val_loss: 0.5700
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7920 - loss: 0.4666 - val_accuracy: 0.8288 - v

/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.5201 - loss: 0.6907 - val_accuracy: 0.2686 - val_loss: 0.7647
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.5965 - loss: 0.6571 - val_accuracy: 0.2781 - val_loss: 0.7208
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5920 - loss: 0.6327 - val_accuracy: 0.7250 - val_loss: 0.6363
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7071 - loss: 0.5648 - val_accuracy: 0.7219 - val_loss: 0.6591
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7665 - loss: 0.5120 - val_accuracy: 0.7036 - val_loss: 0.6903
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7983 - loss: 0.4700 - val_accuracy: 0.6125 - val_loss: 0.7401
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7966 - loss: 0.4530 - val_accuracy: 0.5024 - val_loss: 0.7794
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8214 - loss: 0.4280 - val_accuracy: 0.5721 - v

/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.5085 - loss: 0.6868 - val_accuracy: 0.6573 - val_loss: 0.6753
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5530 - loss: 0.6727 - val_accuracy: 0.6745 - val_loss: 0.6569
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6423 - loss: 0.6427 - val_accuracy: 0.6183 - val_loss: 0.6434
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7008 - loss: 0.6020 - val_accuracy: 0.5496 - val_loss: 0.6720
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7213 - loss: 0.5652 - val_accuracy: 0.6331 - val_loss: 0.7351
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7818 - loss: 0.4803 - val_accuracy: 0.7799 - val_loss: 0.7638
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7832 - loss: 0.4682 - val_accuracy: 0.7947 - val_loss: 1.1668
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.8013 - loss: 0.4596 - val_accuracy: 0.8064 - v

/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.5422 - loss: 0.6779 - val_accuracy: 0.4827 - val_loss: 0.7001
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.5398 - loss: 0.6508 - val_accuracy: 0.5845 - val_loss: 0.7006
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6687 - loss: 0.5988 - val_accuracy: 0.5111 - val_loss: 0.7467
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7693 - loss: 0.5331 - val_accuracy: 0.5194 - val_loss: 0.8983
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8185 - loss: 0.4375 - val_accuracy: 0.5381 - val_loss: 0.9858
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8366 - loss: 0.3902 - val_accuracy: 0.5519 - val_loss: 1.0298
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8352 - loss: 0.3777 - val_accuracy: 0.5069 - val_loss: 1.0441
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8713 - loss: 0.3184 - val_accuracy: 0.4785 - v

/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5301 - loss: 0.6789 - val_accuracy: 0.5303 - val_loss: 0.6930
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5620 - loss: 0.6516 - val_accuracy: 0.4908 - val_loss: 0.6929
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6561 - loss: 0.6280 - val_accuracy: 0.4908 - val_loss: 0.7341
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7089 - loss: 0.5788 - val_accuracy: 0.4847 - val_loss: 0.8076
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.7248 - loss: 0.5613 - val_accuracy: 0.4847 - val_loss: 0.8279
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7369 - loss: 0.5383 - val_accuracy: 0.4901 - val_loss: 0.8311
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7661 - loss: 0.5043 - val_accuracy: 0.4888 - val_loss: 0.8146
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7580 - loss: 0.5059 - val_accuracy: 0.4901 - v

Evaluate the model on the reserved test set

In [63]:
accuracy = []
for train_index, val_index in grpKfold.split(data_array, label_array, groups = group_array):
    
    train_features, train_labels = data_array[train_index], label_array[train_index]
    val_features, val_labels = data_array[val_index], label_array[val_index]

    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, \
                                                                 train_features.shape[-1])).reshape(train_features.shape)
     #basically, that reshape will make the array have a column 19, and row which is the product of the first two elements, thereby making it a 2D data
    val_features = scaler.transform(val_features.reshape(-1,\
                                                               val_features.shape[-1])).reshape(val_features.shape)
    model = cnnmodel()
    model.fit(train_features, train_labels, epochs = 10, batch_size = 100, validation_data = (val_features, val_labels))

    accuracy.append(model.evaluate(val_features, val_labels)[1]) #accuracy is at index 1, loss is at index 0

    break


/Users/folasewaabdulsalam/Signal_Processing/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.5897 - loss: 0.6742 - val_accuracy: 0.3699 - val_loss: 0.7276
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5941 - loss: 0.6513 - val_accuracy: 0.3953 - val_loss: 0.7233
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6600 - loss: 0.6174 - val_accuracy: 0.5745 - val_loss: 0.7740
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7376 - loss: 0.5497 - val_accuracy: 0.6362 - val_loss: 0.7797
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7830 - loss: 0.4960 - val_accuracy: 0.6651 - val_loss: 0.8165
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7899 - loss: 0.4791 - val_accuracy: 0.6788 - val_loss: 0.8331
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8055 - loss: 0.4509 - val_accuracy: 0.7117 - val_loss: 0.8491
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8238 - loss: 0.4361 - val_accuracy: 0.7275 - v

(6970, 1250, 19)